In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
 
def get_website_info(response):
        bs_obj = BeautifulSoup(response, "html.parser")
        if bs_obj:
            all_pages = bs_obj.find_all("table",{"class": "list-categores-table"})
            
            links = set()
            for page in all_pages:
                linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
                for tag in linkTags:
                    link = tag.get('href')
                    print(link)
                
if __name__ == '__main__':
    res = urlopen("https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
    get_website_info(res)

/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Accounting-Finance/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Administrative-Clerical/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Banking/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Brokerage-Operations---Trading/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Business-Strategy/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Campus-Programs-Internships/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Compliance-Legal-Regulatory/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Customer-Service-Client-Service/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Digital-Services/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Facilities-Real-Estate/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Financial-Consultant/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Financial-Planning-Advice/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Human-Resources-Training-Recruiting/
/ListJobs/ByCustom/Schwab-Job-Category/Keyword-Information-Techno

In [74]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import csv

BASE = 'https://www.schwabjobs.com'
 
def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        return dict(zip(catLinks, numPages))
        
def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(BASE + key)
    return pageLinks
        

def scrape_pages(pages):
    for cat in categories:
        cat_page = urlopen(BASE + cat)
        soup = BeautifulSoup(cat_page)
        table = soup.find('table', 'JobListTable')
        links = []
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = BASE + link.get('href')
            scrape(page)
            break
        
def scrape(page):
    
    posting = urlopen(page)

    soup2 =  BeautifulSoup(posting)

    value = soup2.findAll("div", {"class": "desc"})

    enc = 'utf-8'
    with open('freq.csv', 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]

    for val in value:
        st = val.text.split()
        for word in st:
            if(word in arr):
                print(word)
        
if __name__ == '__main__':
    res = urlopen("https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
    categories = get_category(res)
    pages = get_pages(categories)
    scrape_pages(pages)

investing
people
create
driven
purpose
respect
communities
challenge
create
firm
bring
life
employee
guide
invest
adaptable
people
desire
help
clients
established
years
ago
help
work
investing
accessible
enter
innovative
driven
people
help
create
better
responsible
implementing
relationship
management
process
maximize
performance
strategic
assists
meeting
regulatory
compliance
organization
currently
seeking
expand
category
management
strategic
capabilities
space
play
role
responsible
leading
senior
providing
business
partners
expertise
key
technology
responsible
building
business
implementing
action
plans
drive
strategic
position
responsible
strong
relationships
senior
stakeholders
maintaining
category
plans
business
partners
managing
aspects
strategic
well
risk
groups
ensure
tools
support
growth
relationships
suppliers
business
role
requires
building
maintaining
demonstrated
experience
leading
teams
decision
making
strong
communication
effective
analysis
best
practices
role
people
man

company
been
challenging
financial
services
industry
years
mission
help
clients
create
better
future
design
program
engage
directly
culture
challenging
learning
enriching
work
2019
program
organization
meaningful
valuable
experience
learning
real
business
discuss
learning
facilitated
team
discussions
individual
senior
leader
regular
feedback
participate
provide
work
network
interacting
professionals
including
senior
sense
knowledge
enjoy
working
partnerships
collaboration
achieve
common
purpose
innovative
constantly
better
ways
aim
client
expectations
leader
considered
primary
meet
current
program
tracking
2019
work
full
time
location
program
specific
needed
hosting
recognized
industry
culture
embrace
individuality
partner
invest
experience
providing
direction
leadership
robust
well
helping
focus
potential
future
encourage
positions
qualified
opportunity
provide
consistent
experience
program
rising
seniors
year
action
takes
action
employ
qualified
individuals
culture
encourages
applica

investing
people
create
driven
purpose
respect
communities
challenge
create
firm
bring
life
employee
guide
invest
adaptable
people
desire
help
clients
established
years
ago
help
work
investing
accessible
enter
innovative
driven
people
help
create
better
key
project
internal
external
resources
include
ensure
cost
efficient
success
ideal
comfortable
work
effectively
highly
collaborative
strategies
ensure
successful
execution
project
plan
project
including
project
budget
customer
ensure
delivery
supervision
construction
process
include
management
design
development
process
collaborate
internal
change
order
negotiations
contract
sources
market
resources
appropriate
services
inefficiencies
processes
adjustments
improve
vendors
activity
performance
ensure
compliance
objectives
projects
project
costs
ensure
compliance
approved
compliance
strong
relationships
ensure
close
collaboration
communication
decision
enforcement
internal
job
produce
minutes
collaborative
team
years
experience
managing


investing
people
create
driven
purpose
respect
communities
challenge
create
firm
bring
life
employee
guide
invest
adaptable
people
desire
help
clients
established
years
ago
help
work
investing
accessible
enter
innovative
driven
people
help
create
better
asset
management
billion
assets
management
largest
retail
money
wide
variety
products
money
market
mutual
actively
mutual
exchange
traded
funds
asset
allocation
provider
core
investment
mission
helping
fit
critical
profitability
expanding
presence
interested
learning
culture
work
interested
learning
specific
positions
interest
held
location
local
employees
benefits
rewards
work
diverse
client
questions
development
meaningful
time
interest
based
required
potential
opportunities
meeting
requirements
include
event
equal
opportunity
action
committed
policy
provide
equal
opportunities
employees
medical
identity
national
legally
medical
status
status
2019
2019
investing
people
create
driven
purpose
respect
communities
challenge
create
firm
br

investing
people
create
driven
purpose
respect
communities
challenge
create
firm
bring
life
employee
guide
invest
adaptable
people
desire
help
clients
established
years
ago
help
work
investing
accessible
enter
innovative
driven
people
help
create
better
supports
business
operational
management
team
strategy
governance
communication
well
workforce
project
team
organization
deliver
business
predictable
project
execution
effective
solutions
business
problems
technology
strategy
planning
technology
department
capturing
governance
facilitate
implementation
report
tracking
professional
consistent
communication
inside
organization
managing
budget
tracking
reporting
planning
tracking
workforce
engagement
training
creation
executive
management
organizational
reporting
ensure
quality
standards
professional
branding
maintained
policies
minimum
1
3
years
exposure
business
processes
3
years
project
management
experience
communication
ability
build
strong
partnerships
teams
dealing
levels
organizati